In [2]:
import os
from pathlib import Path
import json
import linecache

import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt
import folium 
from shapely.ops import nearest_points
from shapely.geometry import LineString

In [3]:
DATA_PATH = Path('/data/safegraph/safegraph_open_census_data')
PREPROCESSED_DATA_PATH = Path('../../../data/preprocessed/safegraph/safegraph_open_census_data')

In [4]:
# See: https://www.safegraph.com/blog/beginners-guide-to-census
    
table_ids = [
    'B01001e1',   # Total Population
    'B19013e1',   # Median Household Income
]

cbg_field_desc = pd.read_csv(DATA_PATH / 'metadata/cbg_field_descriptions.csv')
cbg_field_desc[cbg_field_desc.table_id.isin(table_ids)]

,table_id,field_full_name,field_level_1,field_level_2,field_level_3,field_level_4,field_level_5,field_level_6,field_level_7,field_level_8
4,B01001e1,SEX BY AGE: Total: Total population -- (Estimate),Sex By Age,Total,Total Population -- (Estimate),NaN,NaN,NaN,NaN,NaN
3094,B19013e1,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,Median Household Income In The Past 12 Months ...,Total,Households -- (Estimate),NaN,NaN,NaN,NaN,NaN


In [6]:
# Only needs to be run once
county_fips_code = '45039'  # Fairfield County, South Carolina

!mkdir -p {PREPROCESSED_DATA_PATH}
census_data_file_names = !ls {DATA_PATH}/data/ | grep [0-9] #  | cut -f 1 -d .  # eliminate .csv suffix
county_directory = PREPROCESSED_DATA_PATH / "data/county" / county_fips_code
!mkdir -p {county_directory}
for file_name in census_data_file_names:
    !touch {county_directory}/{file_name}
    !head -n 1 "{DATA_PATH}/data/{file_name}" > {county_directory}/{file_name}
    !cat "{DATA_PATH}/data/{file_name}" | grep ^{county_fips_code}.*$ >> {county_directory}/{file_name}

^C
^C
^C


In [7]:
cbg_b19 = pd.read_csv(county_directory / 'cbg_b19.csv', dtype={'census_block_group': str})
cbg_b01 = pd.read_csv(county_directory / 'cbg_b01.csv', dtype={'census_block_group': str})
cbg_data = pd.merge(cbg_b01, cbg_b19, on=['census_block_group'])
cbg_data = cbg_data[['census_block_group'] + table_ids]
cbg_data.dropna().head()

,census_block_group,B01001e1,B19013e1
0,450399601001,1101,29250.0
1,450399601002,931,39931.0
2,450399602001,1015,46898.0
3,450399602002,1260,30481.0
4,450399602003,1467,48125.0


In [8]:
cbg_data[['census_block_group'] + table_ids].head()

,census_block_group,B01001e1,B19013e1
0,450399601001,1101,29250.0
1,450399601002,931,39931.0
2,450399602001,1015,46898.0
3,450399602002,1260,30481.0
4,450399602003,1467,48125.0


In [9]:
cbg_data

,census_block_group,B01001e1,B19013e1
0,450399601001,1101,29250.0
1,450399601002,931,39931.0
2,450399602001,1015,46898.0
3,450399602002,1260,30481.0
4,450399602003,1467,48125.0
5,450399603001,1046,68902.0
6,450399603002,890,NaN
7,450399603003,1482,30625.0
8,450399603004,2257,31346.0
9,450399604001,962,26250.0


In [10]:
sum(cbg_data['B01001e1'])

23025

In [11]:
# Census Block Groups

In [16]:
def geojson_for_county(state_abbreviation="SC",
                       county_name="Fairfield County",
                       geojson_path=DATA_PATH / "geometry/cbg.geojson"):
    header = !head -n 5 {geojson_path}
    footer = !tail -n 2 {geojson_path}
    
    # lines to search file for county of interest.
    # must be found by inspection using "tail | head" method below, and checking whether 
    # the state of interest is included.
    # If not included, search up or down via binary search (file is sorted by state)
    # TODO: write the binary search explicitly here, if we need to generalize to other states/counties
    line_start_search = 170000
    line_end_search = 180000
    num_lines = line_end_search - line_start_search
    
    # stream = os.popen(f"""< {geojson_path} tail -n +{line_start_search} | head -n {num_lines} | grep '"State": "{state_abbreviation}", "County": "{county_name}"'  """)
    #stream = os.popen(f"""cat {geojson_path} | tail -n +{line_start_search} | head -n {num_lines} | grep '"State": "{state_abbreviation}", "County": "{county_name}"'  """)
    #county_cbgs = stream.readlines()
    
    county_cbgs = [linecache.getline(str(geojson_path), line_number).strip() for line_number in range(line_start_search, line_end_search)]
    county_cbgs = [line for line in county_cbgs if f'"State": "{state_abbreviation}", "County": "{county_name}"' in line]
    
    print(len(county_cbgs))
    
    # remove final character from last entry in list:
    # a trailing "," that will mess up the JSON parsing
    if county_cbgs[-1][-1] == ',':
        county_cbgs[-1] = county_cbgs[-1][:-1]
    
    return json.loads('\n'.join(header + county_cbgs + footer))
    

In [17]:
geojson = geojson_for_county()

18


In [20]:
len([f['properties']['CensusBlockGroup'] for f in geojson['features']])
#[f['properties']['CensusBlockGroup'] for f in cbgs_json['features']]

18

In [24]:
geojson['features'][0]['properties']

{'StateFIPS': '45',
 'CountyFIPS': '039',
 'TractCode': '960300',
 'BlockGroup': '2',
 'CensusBlockGroup': '450399603002',
 'State': 'SC',
 'County': 'Fairfield County',
 'ClassCode': 'H1'}

In [68]:
m = folium.Map(location=[34.4, -81.1], zoom_start=10.5)

folium.Choropleth(
    geo_data=geojson,
    name='Median Household Income',
    data=cbg_data,
    columns=['census_block_group', 'B19013e1'], # B01001e1, # ['State', 'Unemployment'],
    key_on='feature.properties.CensusBlockGroup',
    fill_color='Greens',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Median Household Income ($)'
).add_to(m)

folium.Choropleth(
    geo_data=geojson,
    name='Population',
    data=cbg_data,
    columns=['census_block_group', 'B01001e1'], # , # ['State', 'Unemployment'],
    key_on='feature.properties.CensusBlockGroup',
    fill_color='Blues',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population'
).add_to(m)

folium.LayerControl().add_to(m)

m